<a href="https://colab.research.google.com/github/quanvu0996/models/blob/master/Movielen_ClusterRecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
# Enviroment setting
import tensorflow as tf
from keras import backend as K
import pprint

import requests
import sys

import numpy as np
import re
from scipy.sparse import csr_matrix, hstack
import pandas as pd
from pandas.api.types import CategoricalDtype
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from joblib import Parallel, delayed

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Tải dữ liệu về (chỉ tải 1 lần, lưu về drive => từ lần sau chỉ việc load lại)
# file_url = 'https://files.grouplens.org/datasets/movielens/ml-20m.zip'
# file_path = "/content/gdrive/MyDrive/RECOMMENDER_STUDIES/data/ml-20m.zip"

# r = requests.get(file_url, stream = True) 

# with open( file_path, "wb") as file: 
#     for block in r.iter_content(chunk_size = 1024*10): 
#         if block: 
#             file.write(block)

In [4]:
# !unzip /content/gdrive/MyDrive/RECOMMENDER_STUDIES/data/ml-20m.zip -d /content/gdrive/MyDrive/RECOMMENDER_STUDIES/data/

In [3]:
itemCol = 'movieId'
userCol = 'userId'

fpath = "/content/gdrive/MyDrive/RECOMMENDER_STUDIES/data/ml-20m"

In [4]:
def fn_year(x):
    x = re.findall('\(\d{4}\)$',x['title'])
    if len(x) >0:
        return x[0][1:5]
    else:
        return None

movies = pd.read_csv(fpath+'/movies.csv')
movies['release_year'] = movies.apply( fn_year , axis=1)
movies["genre_arr"] = movies['genres'].apply(lambda x: x.split('|'))

In [5]:
ratings = pd.read_csv(fpath+'/ratings.csv')

In [8]:
# item-wise split
# test_year = '2005'
# valid_year = '1996'
# ft_movies = movies[movies['release_year']<= valid_year]
# lb_movies = movies[(movies['release_year']> valid_year) & (movies['release_year'] <= test_year)]
# test_movies = movies[movies['release_year']> test_year]

# ft_movies.movieId.count(), lb_movies.movieId.count(), test_movies.movieId.count()

In [6]:
def user_wise_split(df, userCol, itemCol, test_size=0.25):
    all_user = df[userCol].drop_duplicates()
    train_user, test_user = train_test_split(all_user, test_size=test_size)

    train_ratings = df[df[itemCol].isin(train_user)]
    test_ratings = df[df[itemCol].isin(test_user)]
    return train_ratings, test_ratings

train_val, test = train_test_split(ratings, test_size= 0.25)
train, valid = user_wise_split(train_val,userCol, itemCol, test_size=0.25 )
train.head()

,userId,movieId,rating,timestamp
1922206,13005,919,4.0,1254673170
13998030,96693,4963,2.0,1241414967
6675075,45960,469,4.0,840429941
17471727,120842,56782,5.0,1415965499
6888035,47439,44655,4.0,1296774801


MODELING

In [8]:
# Loss function
def Focus2(alpha=0.01):
    def cal_los(y_true, y_pred):
        y_pred = tf.cast(y_pred, tf.float32)
        y_true = tf.cast(y_true, tf.float32)

        positive_true = y_true[y_true!=0]
        positive_pred = y_pred[y_true!=0]
        los1 = K.square(positive_true-positive_pred )

        zero_true = y_true[y_true==0]
        zero_pred = y_pred[y_true==0]
        los2 =alpha*  K.square(zero_true-zero_pred )


        return K.mean( tf.concat([los1, los2], axis=0) )
    return cal_los

y_true = tf.constant([1,0.2,0,1])
y_pred = tf.constant([1,0,5.2,1.005])

Focus2(0.01)(y_true, y_pred)


<tf.Tensor: shape=(), dtype=float32, numpy=0.07760624>

In [142]:
# EVALUATION Function
def checkbox_rating(ratings, split_level = 3):
  "Convert ratings thang 5 điểm về binary"
  return np.where( ratings > split_level, 1, 0)

def pair_iou(r1, r2):
  """Cho 2 bộ: rating1, rating2, dạng onehot => tính IOU"""
  inter = np.multiply(r1,r2) > 0
  union = (r1+r2) > 0

  if np.sum(union) ==0:
    return None
  else:
    inters = np.sum(inter, axis=1)
    unions = np.sum(union, axis=1)
    iou = np.mean(inters[unions>0]/unions[unions>0])
    return iou 

def list_iou(ratings_):
  ratings = ratings_.copy()
  n = ratings.shape[0]
  np.random.shuffle(ratings)

  @delayed
  def f_iou_i(i):
      shift_ratings = np.roll(ratings, i+1)
      iou_i = pair_iou(ratings, shift_ratings)
      return iou_i


  ious = Parallel(n_jobs=-1, backend = "threading")(f_iou_i(i) for i in range(10))
  return np.mean(ious)


def iou_cluster(Y_true, y_cluster_pred):
    ious = []
    for i in np.unique(y_cluster_pred):
        ratings = Y_true[y_cluster_pred == i]
        ious.append(list_iou(ratings))
    return np.mean(ious), ious

SIMPLE CF

In [9]:
# Pivot value, keeping just top_item popular the most
def get_cf_ft(ratings, top_item = 300):
    top_items = list(train.groupby(itemCol).count()["rating"].sort_values(ascending=False)\
                     .head(top_item).index)
    ratings_ = ratings[ratings[itemCol].isin(top_items)]
    cf_fts= ratings_.pivot(index=userCol, columns=itemCol, values = "rating")
    return cf_fts

# Get feature and label, random spliting from a full dataset
def get_ft_lb(ratings, test_size=0.25, top_item=300):
    ft, lb = train_test_split(ratings, test_size= test_size)
    ft_pv = get_cf_ft(ft, top_item)
    lb_pv = get_cf_ft(ft, top_item)

    X = ft_pv.values
    y = lb_pv.reindex(ft_pv.index).values
    return X, y, ft_pv.index
X_cf, y, user_ind = get_ft_lb(train, test_size=0.25)


In [10]:
X_cf.shape, y.shape

((136272, 300), (136272, 300))

In [11]:
# Math model
n_ft = X_cf.shape[1]
n_clusters = 25
n_lb = y.shape[1]
model = tf.keras.Sequential([
          tf.keras.layers.Input(n_ft),
          tf.keras.layers.Dense(n_clusters, activation='softmax'),
        #   tf.keras.layers.Dense(int(n_clusters/4), activation='softmax'),
          tf.keras.layers.Dense(n_lb)
])

model.compile( optimizer='adam', loss= Focus2(0.05))
model.fit(np.nan_to_num(X_cf), np.nan_to_num(y), epochs=5, batch_size=64)

Epoch 1/5
2130/2130 [==============================] - 11s 5ms/step - loss: 0.6520
Epoch 2/5
2130/2130 [==============================] - 6s 3ms/step - loss: 0.3364
Epoch 3/5
2130/2130 [==============================] - 6s 3ms/step - loss: 0.3135
Epoch 4/5
2130/2130 [==============================] - 6s 3ms/step - loss: 0.3051
Epoch 5/5
2130/2130 [==============================] - 6s 3ms/step - loss: 0.2985


In [14]:
lv1 = model.layers[0](np.nan_to_num(X_cf))
lv1_ind = np.argmax(lv1, axis=1)
k=10
print(lv1[k], lv1_ind[k])

for i in np.unique(lv1_ind):
    print(i,":", np.sum(lv1_ind==i))

tf.Tensor(
[0.0000000e+00 3.6926485e-07 5.2853051e-05 6.3609052e-04 0.0000000e+00
 0.0000000e+00 3.9632429e-11 7.7724529e-18 0.0000000e+00 0.0000000e+00
 1.8855225e-16 0.0000000e+00 0.0000000e+00 8.2715757e-02 0.0000000e+00
 0.0000000e+00 5.6058341e-17 0.0000000e+00 4.5427034e-14 9.1464311e-01
 0.0000000e+00 1.9518479e-03 0.0000000e+00 0.0000000e+00 0.0000000e+00], shape=(25,), dtype=float32) 19
1 : 4263
2 : 12602
3 : 5991
6 : 25947
7 : 1028
10 : 7444
13 : 16617
16 : 13778
18 : 8751
19 : 32770
21 : 7081


In [157]:
def iou_cluster(Y_true, y_cluster_pred):
    ious = []
    ns = []
    for i in np.unique(y_cluster_pred):
        ratings = Y_true[y_cluster_pred == i]
        ns.append(ratings.shape[0])
        ious.append(list_iou(ratings))
    ns_arr = np.array(ns)
    ious_arr = np.array(ious)
    return np.sum(ns_arr*ious_arr)/np.sum(ns_arr)

In [158]:
y_checkbox = checkbox_rating(y)
iou = iou_cluster(y_checkbox, lv1_ind)
print(iou)

0.04855981074605795


In [15]:
# lv2 = model.layers[1](lv1)
# lv2_ind = np.argmax(lv2, axis=1)
# for i in np.unique(lv2_ind):
#     print(i,":", np.sum(lv2_ind==i))

# CF + CB model

In [146]:
%%time
def vectorize_arr(movies, itemCol, arrCol, outCol = "genre_vec"):

    out = movies.explode(arrCol)
    columns = out[arrCol].drop_duplicates()
    out["value"]=1

    out = out.pivot(index=itemCol, columns = arrCol, values="value").fillna(0)
    out[outCol] = out.apply(lambda x: np.array(x[columns]), axis=1)
    return out.drop(columns, axis=1)

def cb_ft(ratings, movies,userCol ,itemCol, user_ind=None):
    movie_attr = vectorize_arr(movies,itemCol, arrCol='genre_arr')
    train_ft = ratings.merge(movie_attr, on=itemCol)
    train_ft['rating_attr'] = train_ft['rating'] * train_ft['genre_vec']
    train_ft_byu = train_ft.groupby(userCol).agg({'rating_attr':"mean"})
    if user_ind is not None:
        train_ft_byu = train_ft_byu.reindex(user_ind)
        
    list_ar = [x[0] for x in train_ft_byu.values]
    X_cb = np.stack(list_ar, axis=0)
    return X_cb

X_cfcb = np.hstack([np.nan_to_num(X_cf),
                   cb_ft(train, movies, userCol ,itemCol, user_ind)
                   ])

CPU times: user 1min 24s, sys: 28.7 s, total: 1min 53s
Wall time: 1min 54s


In [147]:
X_cfcb.shape

(136272, 320)

In [148]:
# Math model
n_ft = X_cfcb.shape[1]
n_clusters = 25
n_lb = y.shape[1]
model2 = tf.keras.Sequential([
          tf.keras.layers.Input(n_ft),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Dense(n_clusters, activation='softmax'),
        #   tf.keras.layers.Dense(int(n_clusters/4), activation='softmax'),
          tf.keras.layers.Dense(n_lb)
])

model2.compile( optimizer='adam', loss= Focus2(0.01) )
model2.fit(np.nan_to_num(X_cfcb), np.nan_to_num(y), epochs=5, batch_size=128)

Epoch 1/5
1065/1065 [==============================] - 6s 4ms/step - loss: 0.7661
Epoch 2/5
1065/1065 [==============================] - 5s 5ms/step - loss: 0.2833
Epoch 3/5
1065/1065 [==============================] - 5s 4ms/step - loss: 0.1885
Epoch 4/5
1065/1065 [==============================] - 5s 5ms/step - loss: 0.1665
Epoch 5/5
1065/1065 [==============================] - 5s 4ms/step - loss: 0.1520


In [149]:
lv1_2 = model2.layers[1](model2.layers[0](np.nan_to_num(X_cfcb)))
lv1_ind_2 = np.argmax(lv1_2, axis=1)
print(lv1_2[1])

for i in np.unique(lv1_ind_2):
    print(i,":", np.sum(lv1_ind_2==i))

tf.Tensor(
[4.8878923e-02 9.0589845e-01 3.3117320e-02 5.5799336e-07 1.5746822e-07
 1.2319317e-07 6.5119423e-07 5.5146280e-08 1.4830492e-06 1.6900875e-06
 4.9764600e-07 2.0810081e-07 5.8337026e-08 1.2091854e-02 1.6287899e-07
 1.0708258e-06 1.2668285e-07 1.3519192e-06 1.2261555e-07 1.2528376e-07
 7.5136279e-07 4.2966337e-07 2.1386481e-06 1.3521255e-06 2.5005508e-07], shape=(25,), dtype=float32)
0 : 5308
1 : 65836
2 : 34350
13 : 30778


In [159]:
y_checkbox = checkbox_rating(y)
iou = iou_cluster(y_checkbox, lv1_ind_2)
print(iou)

0.048564875860609744


In [156]:
iou

(0.038822547957990214,
 [0.027230876383575443,
  0.06755595101438942,
  0.045932047038996386,
  0.014571317394999598])

In [23]:
y[lv1_ind==12].shape

(104861, 300)

BASELINE: KMEANS

In [163]:
# KMeans với CF feature
km1 = sklearn.cluster.KMeans()
km1.fit(np.nan_to_num(X_cf), np.nan_to_num(y))

km1_ind = km1.predict(np.nan_to_num(X_cf))

for i in np.unique(km1_ind):
    print(i,":", np.sum(km1_ind==i))

0 : 7619
1 : 17880
2 : 8861
3 : 20986
4 : 11488
5 : 52501
6 : 10872
7 : 6065


In [164]:
# y_checkbox = checkbox_rating(y)
iou = iou_cluster(y_checkbox, km1_ind)
print(iou)

0.048512713248720904


In [165]:
# KMeans với CFCB feature
km2 = sklearn.cluster.KMeans()
km2.fit(np.nan_to_num(X_cfcb), np.nan_to_num(y))

km2_ind = km2.predict(np.nan_to_num(X_cfcb))

for i in np.unique(km2_ind):
    print(i,":", np.sum(km2_ind==i))

0 : 51748
1 : 21094
2 : 17926
3 : 7592
4 : 10956
5 : 6193
6 : 9278
7 : 11485


In [166]:
# y_checkbox = checkbox_rating(y)
iou = iou_cluster(y_checkbox, km2_ind)
print(iou)

0.04851188868586251


EVALUATION

In [143]:
%%time
# y_checkbox = checkbox_rating(y)
iou = iou_cluster(y_checkbox, lv1_ind)
iou

CPU times: user 8.21 s, sys: 535 ms, total: 8.75 s
Wall time: 6.53 s


In [144]:
iou

(0.04908920674218443,
 [0.04986259549791176,
  0.048524567452345894,
  0.04903010774764126,
  0.04918971024521553,
  0.051314851740290414,
  0.04847542653007918,
  0.048861613324826005,
  0.04891807578062232,
  0.04830921677197546,
  0.04891970371456544,
  0.04857540535855545])

In [105]:
k = 10
r1 = y_checkbox[lv1_ind== k]
shift_r1 = np.roll(r1, 10)
iou_i = pair_iou(shift_r1, r1 )
iou_i

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


nan

In [150]:
import gc
gc.collect()

858

0.6666666666666666


In [117]:
iou_i

0.04487990623572415

In [36]:
np.unique(lv1_ind)

array([ 1,  2,  3,  6,  7, 10, 13, 16, 18, 19, 21])